<a href="https://colab.research.google.com/github/Inishpy/DeepLearningWork/blob/main/RaG_exploration_in_langchain.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install langchain
!pip install llama-cpp-python
!pip install pypdf

In [ ]:
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.llms import LlamaCpp
from langchain.callbacks.manager import CallbackManager
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler



from langchain_community.document_loaders import PyPDFLoader

loader = PyPDFLoader("Avagin Updates.pdf")
pages = loader.load_and_split()

In [ ]:
from huggingface_hub import hf_hub_download
model = hf_hub_download(repo_id="TheBloke/phi-2-dpo-GGUF", filename="phi-2-dpo.Q5_K_S.gguf")

In [ ]:
model = "./phi-2-dpo.Q5_K_S.gguf"

In [ ]:
callback_manager = CallbackManager([StreamingStdOutCallbackHandler()])

In [ ]:
llm = LlamaCpp(
    model_path=model,
    temperature=0.75,
    max_tokens=50,
    top_p=1,

    verbose=True,
      n_ctx=2000,  # Verbose is required to pass to the callback manager
)

In [ ]:
from langchain_community.embeddings import OllamaEmbeddings

embeddings = OllamaEmbeddings()

In [ ]:
!pip install faiss-cpu
!pip install pypdf2
!pip install InstructorEmbedding==1.0.1
!pip install sentence-transformers==2.2.2

In [ ]:
from langchain.text_splitter import CharacterTextSplitter
from langchain.embeddings import OpenAIEmbeddings, HuggingFaceInstructEmbeddings
from PyPDF2 import PdfReader
from langchain_community.vectorstores import FAISS

In [ ]:


def get_pdf_text(pdf):
    text = ""

    pdf_reader = PdfReader(pdf)
    for page in pdf_reader.pages:
        text += page.extract_text()
    return text


def get_text_chunks(text):
    text_splitter = CharacterTextSplitter(
        separator="\n",
        chunk_size=1000,
        chunk_overlap=200,
        length_function=len
    )
    chunks = text_splitter.split_text(text)
    return chunks


def get_vectorstore(text_chunks):
    #embeddings = OpenAIEmbeddings()
    embeddings = HuggingFaceInstructEmbeddings(model_name="BAAI/bge-base-en-v1.5")
    vectorstore = FAISS.from_texts(texts=text_chunks, embedding=embeddings)
    return vectorstore




In [ ]:


raw_text = get_pdf_text("Avagin Updates.pdf")

text_chunks = get_text_chunks(raw_text)

vectorstore1 = get_vectorstore(text_chunks)

load INSTRUCTOR_Transformer
max_seq_length  512


In [ ]:
from langchain_community.vectorstores import FAISS
from langchain.text_splitter import RecursiveCharacterTextSplitter


text_splitter = RecursiveCharacterTextSplitter()
documents = text_splitter.split_documents(pages)
vectorstore2 = FAISS.from_documents(documents, embeddings)

In [ ]:
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate

prompt = ChatPromptTemplate.from_template("""Answer the following question based only on the provided context:

<context>
{context}
</context>

Question: {input}""")

document_chain = create_stuff_documents_chain(llm, prompt)

In [ ]:
from langchain.chains import create_retrieval_chain

retriever = vectorstore2.as_retriever()
retrieval_chain = create_retrieval_chain(retriever, document_chain)

In [ ]:

#this is how it works if we dont use chain
docs = retriever.get_relevant_documents("Next weeks task?")

context = ""
for doc in docs:
    #print(doc)
    context += doc.page_content + "\n\n"

augmented_user_input = "Context: " + context + "\n\nQuestion: " + "Next weeks task?" + "\n"

print(augmented_user_input)

In [ ]:
from langchain.chains import create_retrieval_chain

retriever = vectorstore1.as_retriever()
retrieval_chain = create_retrieval_chain(retriever, document_chain)

In [ ]:
docs = retriever.get_relevant_documents("Next weeks task?")

context = ""
for doc in docs:
    #print(doc)
    context += doc.page_content + "\n\n"

augmented_user_input = "Context: " + context + "\n\nQuestion: " + "Next weeks task?" + "\n"

print(augmented_user_input)

Context: Challenges  and Request:  
• GPU  Requirement:  Unfortunately, the fine -tuning and RAG processes require 
significant computational resources, specifically GPUs, which are currently 
unavailable.  
• Domain -Specific  LLMs:  We aim to create eight specialized LLMs, including 
the Finance LLM designated for SatoshiAI. However, GPU access is crucial to 
achieve this objective.  
Urgent  Request:  
To ensure continued progress and meet development goals, we urgently request 
access to one or more GPUs. This will enable us to:  
• Fine-tune  LLMs:  Tailor the LLMs to specific domains, including the Finance 
LLM for SatoshiAI.  
• Run RAG  Scripts:  Enhance the LLMs' performance using the Retrieval -
Augmented Generation technique.  
• Achieve  Domain -Specific  Goals:  Develop the eight planned LLMs, including 
the critical Finance LLM for SatoshiAI.  
 
Android  Integration:  We understand the importance of Android integration and will continue to prioritize

Avagin Updates  
Pr

In [ ]:
retrieval_chain.invoke({"input": "Next weeks task?"})


llama_print_timings:        load time =     862.39 ms
llama_print_timings:      sample time =      41.98 ms /    50 runs   (    0.84 ms per token,  1190.96 tokens per second)
llama_print_timings: prompt eval time =  115260.26 ms /   965 tokens (  119.44 ms per token,     8.37 tokens per second)
llama_print_timings:        eval time =    9275.99 ms /    49 runs   (  189.31 ms per token,     5.28 tokens per second)
llama_print_timings:       total time =  125234.68 ms /  1014 tokens


{'input': 'Next weeks task?',
 'context': [Document(page_content="Challenges  and Request:  \n• GPU  Requirement:  Unfortunately, the fine -tuning and RAG processes require \nsignificant computational resources, specifically GPUs, which are currently \nunavailable.  \n• Domain -Specific  LLMs:  We aim to create eight specialized LLMs, including \nthe Finance LLM designated for SatoshiAI. However, GPU access is crucial to \nachieve this objective.  \nUrgent  Request:  \nTo ensure continued progress and meet development goals, we urgently request \naccess to one or more GPUs. This will enable us to:  \n• Fine-tune  LLMs:  Tailor the LLMs to specific domains, including the Finance \nLLM for SatoshiAI.  \n• Run RAG  Scripts:  Enhance the LLMs' performance using the Retrieval -\nAugmented Generation technique.  \n• Achieve  Domain -Specific  Goals:  Develop the eight planned LLMs, including \nthe critical Finance LLM for SatoshiAI.  \n \nAndroid  Integration:  We understand the importance o

In [ ]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate

prompt_template = ChatPromptTemplate.from_messages(
    [("system", "You are a helpful AI assistant named Envie. You will reply to questions only based on the context that you are provided. If something is out of context, you will refrain from replying and politely decline to respond to the user."), ("user", "{input}")]
)

#llm = ChatNVIDIA(model="mixtral_8x7b")

chain = prompt_template | llm | StrOutputParser()

In [ ]:
for response in chain.stream({"input": augmented_user_input}):
    print(response)